# Main Notebook

File to perform experiments

## Imports

In [ ]:
import os
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm import tqdm

from models.GaussianProcess import GaussianProcess
from acquisition_functions.MES import mes_acq, basic_mes_acq
from acquisition_functions.PESMO import pesmo_acq
from acquisition_functions.MESMO import mesmo_acq
from arguments.arguments import MainArguments

from MOObenchmark import MOOackley, MOOcrossit, MOOquadratic
from utils.calc_pareto import get_pareto_undominated_by, getSetfromFront

from models.MOOEvaluationProblem import MOOEvaluationProblem

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_termination
from pymoo.optimize import minimize

## Algorithm Arguments

In [ ]:
d = 1
    
seed = 0
np.random.seed(seed)

total_iter = 40
initial_iter = 1

lower_bound = -2
upper_bound = 2

lowerBounds = [lower_bound]*d
upperBounds = [upper_bound]*d


## Evaluation

In [ ]:
T=1.2

def evaluation(x):
    return MOOackley(x, c1=-T/2, c2=T/2)

N = 1_001
X = np.linspace(lower_bound,upper_bound,N)
Z = np.zeros((N,2))

O = 2
C = 0

problem = MOOEvaluationProblem(evaluation, lowerBound=lower_bound, upperBound=upper_bound)
algorithm = NSGA2()
res = minimize( problem, 
                algorithm,
                termination = get_termination("n_gen",100))

real_pareto = res.F[np.argsort(res.F[:,0])]

for i in range(N):
    Z[i]=evaluation(X[i])

fig, axs = plt.subplots(1,3, figsize=(15,4))

axs[0].plot(X, Z[:,0], 'b')
axs[0].plot(X, Z[:,1], 'k')
axs[0].plot(res.X, res.F[:,0], 'xr', markersize=5)
axs[0].plot(res.X, res.F[:,1], 'xr', markersize=5)

axs[1].plot(np.reshape(Z,(-1,2))[:,0], np.reshape(Z,(-1,2))[:,1], 'kx')
axs[1].plot(res.F[:,0], res.F[:,1], 'rx')

axs[2].plot(res.F[:,0], res.F[:,1], 'x')
plt.show()

In [ ]:
def random_acq(GP: GaussianProcess, **kwargs):
    while True:
        x_rand = np.random.uniform(GP.lowerBounds, GP.upperBounds, GP.d)
        if GP.X is None or not x_rand in GP.X:
            break
    return x_rand, 0

## N experiments

In [ ]:
savename = "MOOackley/mesmo_acq"

## Kernerl configuration 
k = gpflow.kernels.SquaredExponential()
### GPs Initialization
GP = GaussianProcess(O, C, d, lowerBounds, upperBounds, k, noise_variance=2e-6)

#### Initial samples, at least 1
for l in range(initial_iter):
    ## Get random evaluation point
    while True:
        x_rand = np.random.uniform(lowerBounds[0], upperBounds[0], d)
        if GP.X is None or not x_rand in GP.X:
            break
    ## EVALUATION OF THE OUTSIDE FUNCTION
    y_rand = evaluation(x_rand)
    GP.addSample(x_rand,y_rand)

GP.updateGP()
GP.optimizeKernel()
if False:
    GP.plotSamples()

row = {
    'ns' : len(GP.X),
    'x'  : x_rand,
    'y'  : y_rand
}
metrics = GP.evaluatePareto(real_pareto, showparetos = False, saveparetos = True)
row.update(metrics)
df = pd.DataFrame({k: [v] for k, v in row.items()})

for l in range(total_iter):

    ## Search of the best acquisition function
    start = time.time()
    x_best, acq_best = mes_acq(GP, showplots = False)
    end = time.time()

    ## EVALUATION OF THE OUTSIDE FUNCTION
    y_best = evaluation(x_best)

    ## UPDATE
    GP.addSample(x_best,y_best)     ## Add new sample to the model
    GP.updateGP()                  ## Update data on the GP regressor
    GP.optimizeKernel()             ## Optimize kernel hyperparameters

    ## Evaluate Pareto (distances and hypervolumes)
    row = {
        'ns' : len(GP.X),
        'x'  : x_best,
        'y'  : y_best
    }
    metrics = GP.evaluatePareto(real_pareto, showparetos = False, saveparetos = True)
    row.update(metrics)

    df = df.append(row, ignore_index = True)